# Collapsing rows with the same sample barcode
<hr>

The purpose of this program is to clean the current data set and remove the remaining rows with non-unique barcodes (SHEETBAR). This will be done by combining or "collapsing" the rows witht he same barcode. While filtering for  the samples taken at the surface of the river (CALCZCD == "SF") and removing samples with bad QF codes accounted for a majority of the rows with identical barcodes, there are still several rows that need to be removed.
 

It is important that our data sets consists only of unqiue barcodes because it removes issues when interpolating the data and predicting our missing continuous variable values. For example, if two samples have the same barcode, then their latitude and longitude points are going to be equal. Say one of these two records has a missing TP value. When the interpolation algorithm tries to predict the missing TP value, it will search for the closest samples and create a new TP value as a linear combination of those closest points. Since we have two identcal lat and long points, the distance will be 0 and thus, by definition of our weights in the linear combination, we will be diving by 0.
 
In each instance of multiple rows with the same SHEETBAR, there will be some combinations of sample and NA values for each continuous variable. Since our goal is combine it to combine the rows with the same SHEETBAR, we will find the average for each column, excluding the DATE, STRATUM, LOCATCD, LATITUDE and LONGITUDE columns.

The data set at this point should have already been filtered for the surface samples and bad QF codes.


In [51]:
# Import libraries
import pandas as pd
from geopy import distance
pd.set_option('display.max_columns', None)
import numpy as np

In [57]:
# The data at this point will have already been filtered by its QF code and by its CALCZCD
# The data set should consist of only surface level points 
data = pd.read_csv("../LTRM data/water_data_filtered.csv")


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (20,21,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [58]:
# Filter out the continous variables that we want
# Gets rid of the columns with QF codes
data = data[['SHEETBAR','TN','TP','TEMP','DO','TURB','COND','VEL','SS','WDP','CHLcal','SECCHI', 'LONGITUDE', 'LATITUDE', 'DATE', 'FLDNUM', 'LOCATCD', 'STRATUM']]

In [59]:
# We know based on inspection that there may be negative TP and TN values. We are going to remove these samples
data = data.drop(data.index[data['TP'] < 0])
data = data.drop(data.index[data['TN'] < 0])
print(data.shape)

(105153, 18)


In [66]:
# Determine how many unique SHEETBAR codes there are and how many times each code occurs
# The number of rows in our final dataframe should be the same number of rows in unique_sheetbars
unique_sheetbars = data.groupby(['SHEETBAR']).size()
print(unique_sheetbars.shape)

(105147,)


In [67]:
# Rename the column to "count" 
unique_sheetbars = pd.DataFrame(unique_sheetbars, columns = ['count']).reset_index()

# Filter out the sheetbar codes that have multiple rows 
duplicate_sheetbars = unique_sheetbars[unique_sheetbars['count'] > 1]
print(duplicate_sheetbars.shape)
print(duplicate_sheetbars)


(4, 2)
        SHEETBAR  count
22957   42009244      2
97418   46015274      2
104734  47000506      3
104840  47000678      3


In [44]:
# Create empty dataframe that will store the new collapsed data set
collapsed_data = pd.DataFrame()

In [46]:
# Since we only need to examine the data that has duplicate barcodes, we can set aside the unique barcodes
collapsed_data = data[-pd.Series(data["SHEETBAR"]).isin(duplicate_sheetbars['SHEETBAR'])]
print(collapsed_data.shape)

# Store the rows with the repeated sheetbars called data_dups 
data_dups = data[pd.Series(data['SHEETBAR']).isin(duplicate_sheetbars['SHEETBAR'])]
print(data_dups.shape)


(105143, 18)
(10, 18)


In [31]:
# Create a separate dataframe that stores the date, stratum, fldnum, lat, long, and location code of each sample
# Drops duplicates so that we will only have one set of values per barcode
identifiers = pd.DataFrame(data_dups, columns = ['SHEETBAR','DATE', 'STRATUM', 'LOCATCD', 'FLDNUM', 'LATITUDE', 'LONGITUDE']).drop_duplicates()

In [32]:
# Remove the identifier columns so that their averages are not taken
data_dups = data_dups[['SHEETBAR','TN','TP','TEMP','DO','TURB','COND','VEL','SS','WDP','CHLcal','SECCHI']]

In [33]:
# Find the average of each continuous variable by barcode
# If there is only na values, na will be returned
# For any number of values per variable, the average will be found
temp = data_dups.groupby(['SHEETBAR'], as_index = False).mean()
print(temp.shape)

(4, 12)


In [34]:
# Adds the indentifiers to the collapsed data by the sheetbar
temp = temp.merge(identifiers, on = ['SHEETBAR'])
print(temp.shape)

(4, 18)


In [50]:
# Add the averaged rows to our final dataframe
collapsed_data = collapsed_data.append(temp, ignore_index = True)


In [36]:
# We don't need thi
#collapsed_data = collapsed_data.reset_index(drop = True)

In [37]:
collapsed_data.to_csv("../LTRM data/cleaned_data.csv")